## 基于Transformer的命名实体识别

#### Step1. 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification,TrainingArguments, Trainer, DataCollatorForTokenClassification
import evaluate
from datasets import load_dataset

2023-09-30 17:17:51.299699: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Step2. 加载数据集

In [ ]:
ner_datasets = load_dataset("peoples_daily_ner", cache_dir="./dataset")

In [ ]:
# ner_datasets["train"][0]
# ner_datasets["train"].features
label_list = ner_datasets["train"].features["ner_tags"].feature_names
# label_list

#### Step3. 数据预处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
tokenizer(ner_datasets["train"][0]["tokens"], is_split_into_words=True) # 对于已经做好tokenize的数据，要指定is_split_into_words参数为True

In [ ]:
res = tokenizer("interesting word")
# res
res.word_ids()

In [2]:
# 借助word_ids 实现标签映射
def process_function(examples):
    tokenized_examples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i,label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_examples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_examples["labels"] = labels
    return tokenized_examples

In [ ]:
tokenized_datasets = ner_datasets.map(process_function, batched=True)
tokenized_datasets

In [ ]:
print(tokenized_datasets["train"][0])

#### Step4. 创建模型

In [ ]:
# 对于所有的非二分类任务，切记要指定num_labels,否则会有device错误
model = AutoModelForTokenClassification.from_pretrained("hfl/chinese-macbert-base", num_labels=len(label_list))

#### Step5. 创建评估函数

In [ ]:
seqeval = evaluate.load("seqeval")
seqeval

In [ ]:
import numpy as np
def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    # 将id转换为原始的字符串类型的标签
    true_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")

    return {
        "f1":result["overall_f1"]
    }

#### Step6. 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="model_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=50,
    num_train_epochs=3
)

#### Step7. 创建训练器

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

#### Step8. 模型训练

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

#### Step9. 模型预测

In [ ]:
from transformers import pipeline
# 使用pipeline进行推理，要指定id2label
model.config.id2label = {idx:label for idx,label in enumerate(label_list)}
# 如果模型是基于GPU训练的，推理时要指定device
# 对于NER任务，要指定aggregation_strategy为simple,得到具体的实体的结果，而不是token结果
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, device=0,aggregation_strategy="simple")
ner_pipe

In [ ]:
res = ner_pipe("小明在北京上班")

In [ ]:
# 根据start和end取实际的结果
ner_result = {}
sentience = "小明在北京上班"
for r in res:
    if r["entity_group"] not in ner_result:
        ner_result[r["entity_group"]] = []
    ner_result[r["entity_group"]].append(sentience[r["start"]:r["end"]])
ner_result

#### Step10. 模型保存

In [ ]:
trainer.save_model("model_for_ner")